# Imports

In [1]:
import os, sys, pickle, math
import numpy as np
from glob import glob
from shutil import copyfile
from PIL import ImageFile


from sklearn.cross_validation import KFold
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Lambda
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation, Input, GlobalAveragePooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.utils.data_utils import get_file
from keras.preprocessing import image
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from sklearn.metrics import log_loss
from keras import backend as K
from keras.applications.inception_v3 import InceptionV3

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


# Paths

In [2]:
cur_dir=os.getcwd()

In [3]:
data_dir=cur_dir+'/data/'
#data_dir=cur_dir+'/data/sample/'
test_dir=cur_dir+'/data/test/'
local_model_path=os.path.dirname(cur_dir)+'/models/'

In [4]:
train_dir=data_dir+'train/'
valid_dir=data_dir+'valid/'
save_dir=data_dir+'save/'

type1_dir=train_dir+'Type_1/'
type2_dir=train_dir+'Type_2/'
type3_dir=train_dir+'Type_3/'

In [26]:
sample_data_dir=data_dir+'sample/'
sample_train_dir=sample_data_dir+'train/'
sample_valid_dir=sample_data_dir+'valid/'
sample_save_dir=sample_data_dir+'save/'

sample_type1_dir=sample_train_dir+'Type_1/'
sample_type2_dir=sample_train_dir+'Type_2/'
sample_type3_dir=sample_train_dir+'Type_3/'

# Directories

In [6]:
%cd $data_dir
%mkdir -p valid/Type_1
%mkdir -p valid/Type_2
%mkdir -p valid/Type_3
%mkdir results
%mkdir save
%mkdir -p sample/train/Type_1
%mkdir -p sample/train/Type_2
%mkdir -p sample/train/Type_3
%mkdir -p sample/test
%mkdir -p sample/valid/Type_1
%mkdir -p sample/valid/Type_2
%mkdir -p sample/valid/Type_3
%mkdir -p sample/results
%mkdir -p sample/save
%mkdir -p test/unknown

/tmp/working/Intel/data


# Configuration

In [5]:
conf = dict()
conf['sample_size'] = 0.33
conf['val_split'] = 0.2
conf['batch_size'] = 64
conf['nb_epoch'] = 1
conf['patience'] = 3
conf['image_size'] = (299, 299, 3)

# Setup Directories

In [8]:
%cd $type1_dir
g = glob('*.jpg')
shuf=np.random.permutation(g)
for i in range(int(len(g)*conf['val_split'])): os.rename(shuf[i], valid_dir+ 'Type_1/' + shuf[i])

/tmp/working/Intel/data/train/Type_1


In [9]:
%cd $type2_dir
g = glob('*.jpg')
shuf=np.random.permutation(g)
for i in range(int(len(g)*conf['val_split'])): os.rename(shuf[i], valid_dir + 'Type_2/' + shuf[i])

/tmp/working/Intel/data/train/Type_2


In [10]:
%cd $type3_dir
g = glob('*.jpg')
shuf=np.random.permutation(g)
for i in range(int(len(g)*conf['val_split'])): os.rename(shuf[i], valid_dir + 'Type_3/' + shuf[i])

/tmp/working/Intel/data/train/Type_3


In [11]:
%cd $type1_dir
g = glob('*.jpg')
shuf=np.random.permutation(g)
for i in range(int(len(g)*conf['sample_size'])): copyfile(shuf[i], sample_type1_dir + shuf[i])

/tmp/working/Intel/data/train/Type_1


In [12]:
%cd $type2_dir
g = glob('*.jpg')
shuf=np.random.permutation(g)
for i in range(int(len(g)*conf['sample_size'])): copyfile(shuf[i], sample_type2_dir + shuf[i])

/tmp/working/Intel/data/train/Type_2


In [13]:
%cd $type3_dir
g = glob('*.jpg')
shuf=np.random.permutation(g)
for i in range(int(len(g)*conf['sample_size'])): copyfile(shuf[i], sample_type3_dir + shuf[i])

/tmp/working/Intel/data/train/Type_3


In [14]:
%cd $sample_type1_dir
g = glob('*.jpg')
shuf=np.random.permutation(g)
for i in range(int(len(g)*conf['val_split'])): os.rename(shuf[i], sample_valid_dir+ 'Type_1/' + shuf[i])

/tmp/working/Intel/data/sample/train/Type_1


In [15]:
%cd $sample_type2_dir
g = glob('*.jpg')
shuf=np.random.permutation(g)
for i in range(int(len(g)*conf['val_split'])): os.rename(shuf[i], sample_valid_dir+ 'Type_2/' + shuf[i])

/tmp/working/Intel/data/sample/train/Type_2


In [16]:
%cd $sample_type3_dir
g = glob('*.jpg')
shuf=np.random.permutation(g)
for i in range(int(len(g)*conf['val_split'])): os.rename(shuf[i], sample_valid_dir+ 'Type_3/' + shuf[i])

/tmp/working/Intel/data/sample/train/Type_3


In [21]:
%cd $test_dir
%mv *.jpg unknown/

/tmp/working/Intel/data/test
mv: cannot stat ‘*.jpg’: No such file or directory


In [7]:
%cd $cur_dir

/tmp/working/Intel


# Utils

In [13]:
def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, batch_size=64, class_mode='categorical',
                target_size=(224,224)):
    return gen.flow_from_directory(dirname, target_size=target_size,
            class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)



# Load Data

In [9]:
gen=image.ImageDataGenerator(preprocessing_function=preprocess_input)

In [14]:
batches_train=get_batches(train_dir, target_size=conf['image_size'][0:2])
batches_valid=get_batches(valid_dir, target_size=conf['image_size'][0:2])

Found 1185 images belonging to 3 classes.
Found 296 images belonging to 3 classes.


In [15]:
train_labels=to_categorical(batches_train.classes)
valid_labels=to_categorical(batches_valid.classes)

In [9]:
train_labels.shape

(1185, 3)

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)

In [25]:
model=Sequential()
model.add(GlobalAveragePooling2D(input_shape=(8, 8, 2048)))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))

In [2]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

Object `predict_generator` not found.


In [38]:
train_base=base_model.predict_generator(batches_train, math.ceil(batches_train.n/conf['batch_size']))
valid_base=base_model.predict_generator(batches_valid, math.ceil(batches_valid.n/conf['batch_size']))

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/keras/engine/training.py", line 606, in data_generator_task
    generator_output = next(self._generator)
  File "/usr/local/lib/python3.5/dist-packages/keras/preprocessing/image.py", line 727, in __next__
    return self.next(*args, **kwargs)
  File "/usr/local/lib/python3.5/dist-packages/keras/preprocessing/image.py", line 960, in next
    target_size=self.target_size)
  File "/usr/local/lib/python3.5/dist-packages/keras/preprocessing/image.py", line 330, in load_img
    img = img.resize(wh_tuple)
  File "/usr/local/lib/python3.5/dist-packages/PIL/Image.py", line 1630, in resize
    self.load()
  File "/usr/local/lib/python3.5/dist-packages/PIL/ImageFile.py", line 231, in load
    "(%d 

ValueError: Cannot feed value of shape () for Tensor 'lambda_2_input:0', which has shape '(?, 224, 224, 3)'

In [16]:
train_base=np.load(save_dir+'X2_train_iv3_299.npy')
valid_base=np.load(save_dir+'X2_valid_iv3_299.npy')

In [17]:
train_base.shape

(1185, 8, 8, 2048)

In [26]:
sgd = SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)

In [27]:
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [31]:
model.fit(train_base, train_labels, batch_size=conf['batch_size'], epochs=20, validation_data=(valid_base, valid_labels))

Train on 1185 samples, validate on 296 samples
Epoch 1/20
1185/1185 [==============================] - 1s - loss: 0.2599 - acc: 0.9637 - val_loss: 1.4588 - val_acc: 0.4324
Epoch 2/20
1185/1185 [==============================] - 1s - loss: 0.2342 - acc: 0.9747 - val_loss: 1.4631 - val_acc: 0.4088
Epoch 3/20
1185/1185 [==============================] - 1s - loss: 0.2246 - acc: 0.9679 - val_loss: 1.6281 - val_acc: 0.3176
Epoch 4/20
1185/1185 [==============================] - 1s - loss: 0.1974 - acc: 0.9831 - val_loss: 1.4914 - val_acc: 0.4662
Epoch 5/20
1185/1185 [==============================] - 1s - loss: 0.1840 - acc: 0.9789 - val_loss: 1.6230 - val_acc: 0.4899
Epoch 6/20
1185/1185 [==============================] - 1s - loss: 0.1679 - acc: 0.9907 - val_loss: 1.5377 - val_acc: 0.4561
Epoch 7/20
1185/1185 [==============================] - 1s - loss: 0.1548 - acc: 0.9916 - val_loss: 1.5777 - val_acc: 0.3986
Epoch 8/20
1185/1185 [==============================] - 1s - loss: 0.1462 - ac

In [68]:
len(train_base[0])

4096